In [1]:
train_dir = "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/train"
test_dir = "/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data/test"

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications import ResNet152V2, DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, InputLayer, BatchNormalization, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from keras_self_attention import SeqSelfAttention

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

2024-02-11 15:54:51.006694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 15:54:51.006776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 15:54:51.129810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [4]:
img_width, img_height = 400,400
BATCH_SIZE_PER_REPLICA = 8
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
epochs = 200
num_classes = 196

In [5]:
train_datagen = ImageDataGenerator( rescale=1./255, rotation_range=30, width_shift_range=0.25, 
                                   height_shift_range=0.25, shear_range=0.25, zoom_range=0.25, 
                                   horizontal_flip=True, fill_mode='nearest' )
        

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory( train_dir, 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, class_mode='categorical', 
                                                    shuffle=True)

test_generator = test_datagen.flow_from_directory( test_dir, 
                                                  target_size=(img_width, img_height), 
                                                  batch_size=batch_size, class_mode='categorical', 
                                                  shuffle=False )

Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model

from tensorflow.keras.layers import *

In [7]:
import tensorflow as tf
from keras.initializers import HeNormal
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = tf.keras.layers.Dense(channel // ratio,
                                             activation = 'sigmoid',
                                             kernel_initializer='he_normal',
                                             use_bias=True,
                                             bias_initializer='zeros')
    shared_layer_two = tf.keras.layers.Dense(channel,
                                             activation = 'sigmoid',
                                             kernel_initializer='he_normal',
                                             use_bias=True,
                                             bias_initializer='zeros')
    
    avg_pool = tf.reduce_mean(input_feature, axis=[1, 2], keepdims=True)
    max_pool = tf.reduce_max(input_feature, axis=[1, 2], keepdims=True)

    avg_pool = shared_layer_one(avg_pool)
    max_pool = shared_layer_one(max_pool)

    avg_pool = shared_layer_two(avg_pool)
    max_pool = shared_layer_two(max_pool)

    attention = tf.math.sigmoid(avg_pool + max_pool)

    return attention

def spatial_attention(input_feature):
    #kernel_size = 7
    kernel_size = 3
    avg_pool = tf.reduce_mean(input_feature, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(input_feature, axis=-1, keepdims=True)

    concat = tf.concat([avg_pool, max_pool], axis=-1)
    conv = tf.keras.layers.Conv2D(filters=1,
                                  kernel_size=kernel_size,
                                  strides=1,
                                  padding='same',
                                  kernel_initializer='he_normal',
                                  activation = 'sigmoid'
                                  )(concat)

    return conv

def cbam_module(input_feature):
    channel_att = channel_attention(input_feature)
    spatial_att = spatial_attention(input_feature)

    cbam_feature = input_feature * channel_att * spatial_att

    return cbam_feature




In [8]:
def base_model(input_shape):
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer._name = layer.name + str("_Densenet")
    return base_model


In [9]:
def base_model1(input_shape):
    base_model1 = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model1.layers:
        layer._name = layer.name + str("ResNet50V2")
    return base_model1

In [10]:
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
with strategy.scope():    
    input_shape=(400,400,3)
    input_tensor = Input(shape=input_shape)
    
    bm_1 = base_model(input_shape)
    bm_2 = base_model1(input_shape)
    y = bm_1(input_tensor)
    z = bm_2(input_tensor)
    
    y2 = GlobalMaxPooling2D(keepdims=True)(y) 
    y2 = Dropout(0.015)(y2)
    
    z2 = GlobalMaxPooling2D(keepdims=True)(z) 
    z2 = Dropout(0.015)(z2)


    x = tf.keras.layers.Concatenate()([y2,z2])

    print(x.shape) 
    
    x = Conv2D(512, (3, 3), padding='same', activation='relu',name='conv2', kernel_regularizer=tf.keras.regularizers.l2(0.01), kernel_initializer="he_normal")(x)
    x = Dropout(0.015)(x)
    
    l = GlobalMaxPooling2D(keepdims=True)(x)
    z1 = cbam_module(l)
    
    a = Conv2D(512, (3, 3), padding='same', activation='relu',name='conv3', kernel_regularizer=tf.keras.regularizers.l2(0.01), kernel_initializer="he_normal")(z1)
    a = BatchNormalization()(a)
    a = Dropout(0.015)(a)
    a = GlobalMaxPooling2D()(a)
    #a = Reshape((1, 1, 512))(a)
    
    #a = Dense(num_classes, activation='softmax')(a)
    #a = GlobalAveragePooling2D()(a)
    #preds = tf.keras.layers.Reshape((num_classes,))(a)
    
    preds = Dense(units=196, activation='softmax')(a)
    
    model = tf.keras.Model(inputs=input_tensor, outputs=preds)
    model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'categorical_crossentropy',metrics=['accuracy','mse'])

model.summary




94668760/94668760 [==============================] - 1s 0us/step
(None, 1, 1, 3968)


<bound method Model.summary of <keras.src.engine.functional.Functional object at 0x7fc43e61c2e0>>

In [11]:
#tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [12]:
#tf.config.experimental_run_functions_eagerly(True)


In [13]:
for layer in model.layers:
    layer.trainable = True

In [14]:

checkpoint = ModelCheckpoint('DenseNet201_Modified_ThreeSoftAtten&Dropouts_Stanford.h5', monitor='val_accuracy', 
                              save_best_only = True, save_weights_only = True, mode ='max', verbose = 2)

early_stop = EarlyStopping(monitor = 'val_accuracy', patience = 10, mode = 'max', verbose = 2)

reduce_learning_rate=ReduceLROnPlateau(monitor = "val_accuracy", factor = 0.1, patience = 4, verbose = 2)

In [15]:
import time
start_time = time.time()

history=model.fit(train_generator, epochs = epochs, validation_data = test_generator,callbacks = [checkpoint,early_stop,reduce_learning_rate])

end_time = time.time()
preprocessing_time = end_time - start_time

print("Preprocessing completed. Time taken:", preprocessing_time, "seconds")

Epoch 1/200


2024-02-11 15:57:13.609439: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1707667049.130529     101 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1018/1018 [==============================] - ETA: 0s - loss: 7.4628 - accuracy: 0.0777 - mse: 0.0050
Epoch 1: val_accuracy improved from -inf to 0.16963, saving model to DenseNet201_Modified_ThreeSoftAtten&Dropouts_Stanford.h5
1018/1018 [==============================] - 789s 615ms/step - loss: 7.4628 - accuracy: 0.0777 - mse: 0.0050 - val_loss: 4.7957 - val_accuracy: 0.1696 - val_mse: 0.0048 - lr: 1.0000e-04
Epoch 2/200
1018/1018 [==============================] - ETA: 0s - loss: 3.7327 - accuracy: 0.3725 - mse: 0.0042
Epoch 2: val_accuracy improved from 0.16963 to 0.50578, saving model to DenseNet201_Modified_ThreeSoftAtten&Dropouts_Stanford.h5
1018/1018 [==============================] - 596s 585ms/step - loss: 3.7327 - accuracy: 0.3725 - mse: 0.0042 - val_loss: 2.7783 - val_accuracy: 0.5058 - val_mse: 0.0034 - lr: 1.0000e-04
Epoch 3/200
1018/1018 [==============================] - ETA: 0s - loss: 2.3294 - accuracy: 0.6318 - mse: 0.0029
Epoch 3: val_accuracy improved from 0.50578 to